In [1]:
classDiagram
    direction LR

    %% Main function class
    class Add-DbDocument {
        +LiteDatabase Database
        +string Collection
        +PSCustomObject Data
        +string[] IgnoreFields
        +bool NoVersionUpdate
        +bool NoTimestampUpdate
        +PSCustomObject execute() 
    }

    %% Core database structure
    class LiteDatabase {
        +string Name
        +Collection GetCollection(string name)
    }

    class Collection {
        +string Name
        +void Insert(PSCustomObject document)
        +PSCustomObject FindOne(string VersionId)
        +PSCustomObject FindLatest(string BundleId)
        +int Count(string BundleId)
    }

    class PSCustomObject {
        +Properties Name-Value Dictionary
    }

    %% Document & Versioning System
    class Document {
        +string _id
        +string BundleId
        +string ContentMark
        +string VersionId
        +int $ObjVer
        +long UTC_Created
        +long UTC_Updated
    }

    class Bundle {
        +Guid BundleId
        +Document[] Versions
    }

    class ContentMark {
        +string Value
        +static string Compute(PSCustomObject Data, string[] IgnoreFields)
    }

    class VersionId {
        +string Value
        +static string Compute(ContentMark ContentMark, BundleId bundleId)
    }

    %% Supporting Operations
    class Get-DataHash {
        +string Hash
        +static string Compute(PSCustomObject Data, string[] FieldsToIgnore)
    }

    class Get-DbDocumentByVersion {
        +PSCustomObject Get(Database, string Collection, string VersionId)
    }

    class Get-DbDocumentVersion {
        +PSCustomObject Get(Database, string Collection, string VersionId, bool Latest)
    }

    class Get-DbDocumentVersionsByBundle {
        +int Count(Database, string Collection, string BundleId)
    }

    class New-DbVersionIdRef {
        +PSCustomObject CreateReference(PSCustomObject DbDocument, string Collection, string RefCollection)
    }

    class Add-LiteData {
        +void Insert(string Collection, PSCustomObject Data)
    }

    class Set-LiteData {
        +void Update(string Collection, PSCustomObject Data)
    }

    %% Relationships
    Add-DbDocument --> LiteDatabase : "Uses"
    Add-DbDocument --> Collection : "Targets"
    Add-DbDocument --> PSCustomObject : "Processes"
    Add-DbDocument --> Get-DataHash : "Computes Hash"
    Add-DbDocument --> Get-DbDocumentByVersionId : "Checks Version"
    Add-DbDocument --> Get-DbDocumentVersion : "Fetches Latest"
    Add-DbDocument --> Get-DbDocumentVersionsByBundle : "Counts Versions"
    Add-DbDocument --> New-DbVersionIdRef : "Handles Versioning"
    Add-DbDocument --> Add-LiteData : "Inserts Document"
    Add-DbDocument --> Set-LiteData : "Updates Document"

    Document --> "1" ContentMark : "Has"
    Document --> "1" VersionId : "Has"
    Document --> "1" Bundle : "Belongs To"
    Bundle --> "*" Document : "Contains Versions"

    ContentMark ..> Get-DataHash : "Computed Using"
    VersionId ..> ContentMark : "Derived From"
    VersionId ..> BundleId : "Derived From"
